## Exploration of airport delay data

This notebook will explore the airport delays web scraped data and use various models to make predictions about future delays - 1 day ahead. Predictions will be based on historical delay data and weather data at each airport.

## Download data locally

The webscraper creates a departures and arrivals CSV for each airport daily.
The following code will download and merge all the CSVs into 2 files: arrivals and departures

In [26]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io
import os

# Your connection string
connect_str = ""

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Get a reference to the container
container_client = blob_service_client.get_container_client("aviation499")


def get_df(arrive_departure, csv_filename):
    # Check if the CSV file already exists
    if os.path.exists(csv_filename):
        # Load the CSV file into a DataFrame
        df = pd.read_csv(csv_filename)
    else:
        blob_list = container_client.list_blobs(name_starts_with=arrive_departure)
        combined_df = pd.DataFrame()

        # Iterate over each blob in the folder
        for blob in blob_list:
            # Check if the blob is a CSV file
            if blob.name.endswith(".csv"):
                # Download the CSV file
                blob_data = container_client.get_blob_client(blob.name).download_blob().readall()
                
                # Convert the blob data to a DataFrame
                df = pd.read_csv(io.StringIO(blob_data.decode("utf-8")))
                
                # Extract the airport name and date from the blob name
                _, date, airport_name = blob.name.split("/")
                
                # Add airport name and date columns to the DataFrame
                df['Airport Name'] = airport_name.replace(".csv", "")
                df['Date'] = date
                
                combined_df = pd.concat([combined_df, df], ignore_index=True)
        
        # Save the combined DataFrame as a CSV file
        combined_df.to_csv(csv_filename, index=False)
        
        # Assign the DataFrame to the variable
        df = combined_df
    
    return df


# Specify the filenames for arrivals and departures
arrivals_csv_filename = "arrivals.csv"
departures_csv_filename = "departures.csv"

# Get the arrivals and departures data as DataFrame
combined_df_arrivals = get_df("arrivals", arrivals_csv_filename)
combined_df_departures = get_df("departures", departures_csv_filename)

# Print the head of the arrivals and departures data
#print(combined_df_arrivals.head())
#print(combined_df_departures.head())





## Pre-processing and cleaning

- Find null values
- Drop redundant columns,
- Value counts for flight status code - diverted and cancelled become delayed, landed is on time (everything is disruption)

In [27]:
# Find and count null values in the arrivals DataFrame
arrivals_null_counts = combined_df_arrivals.isnull().sum()
print("Null value counts in arrivals:")
print(arrivals_null_counts)

# Find and count null values in the departures DataFrame
departures_null_counts = combined_df_departures.isnull().sum()
print("Null value counts in departures:")
print(departures_null_counts)

Null value counts in arrivals:
origin                0
arrive_depart         0
flight                0
airline              48
terminal          37724
status                0
airport_code          0
Ingestion Date        0
Airport Name          0
Date                  0
dtype: int64
Null value counts in departures:
origin                0
arrive_depart         0
flight                0
airline              32
terminal          28899
status                0
airport_code          0
Ingestion Date        0
Airport Name          0
Date                  0
dtype: int64


In [28]:
# Remove rows with null values in the "airline" column in the arrivals DataFrame
combined_df_arrivals = combined_df_arrivals.dropna(subset=['airline'])

# Remove rows with null values in the "airline" column in the departures DataFrame
combined_df_departures = combined_df_departures.dropna(subset=['airline'])


# Drop the specified columns
combined_df_departures = combined_df_departures.drop(['flight', 'airline', 'arrive_depart','terminal','Ingestion Date'], axis=1)
combined_df_arrivals = combined_df_arrivals.drop(['flight', 'airline', 'arrive_depart','terminal','Ingestion Date'], axis=1)

combined_df_departures

In [57]:
# Count unique status types in the arrivals DataFrame
arrivals_status_counts = combined_df_arrivals['status'].value_counts()
print("Unique status types in arrivals:")
print(arrivals_status_counts)

# Count unique status types in the departures DataFrame
departures_status_counts = combined_df_departures['status'].value_counts()
print("Unique status types in departures:")
print(departures_status_counts)

Unique status types in arrivals:
status
Landed - On-time [+]     304389
Landed - Delayed [+]      91236
Landed - DIVERTED [+]      2603
Landed [+]                    1
Name: count, dtype: int64
Unique status types in departures:
status
Landed - On-time [+]     237212
Landed - Delayed [+]      78020
Landed - DIVERTED [+]       972
Landed [+]                   12
Name: count, dtype: int64


In [30]:
# Define a mapping dictionary for encoding the status values
status_mapping = {
    'Landed - On-time [+]': 0,
    'Landed [+]': 0,
    'Landed - Delayed [+]': 1,
    'Landed - DIVERTED [+]': 1
}

# Encode the "status" column in the arrivals DataFrame
combined_df_arrivals['status_encoded'] = combined_df_arrivals['status'].map(status_mapping)

# Encode the "status" column in the departures DataFrame
combined_df_departures['status_encoded'] = combined_df_departures['status'].map(status_mapping)\

print(combined_df_departures['status_encoded'])

0         0
1         0
2         1
3         0
4         0
         ..
316243    0
316244    0
316245    1
316246    0
316247    0
Name: status_encoded, Length: 316216, dtype: int64


In [31]:
# Format origin column for airport

# Create the new column 'origin_code'
combined_df_departures['origin'] = combined_df_departures['origin'].str.split('\n').str[1].str.strip('()')
combined_df_arrivals['origin'] = combined_df_arrivals['origin'].str.split('\n').str[1].str.strip('()')

#combined_df_arrivals['origin_code']


In [32]:
# Convert the "Date" column in the arrivals DataFrame to datetime type
combined_df_arrivals['Date'] = pd.to_datetime(combined_df_arrivals['Date'], format='%Y%m%d')

# Convert the "Date" column in the departures DataFrame to datetime type
combined_df_departures['Date'] = pd.to_datetime(combined_df_departures['Date'], format='%Y%m%d')

#combined_df_departures["Date"]

In [40]:
# Removing code sharing parnters from airline column
# First airline in string is the airline itself - creating new column with only first

combined_df_arrivals['airline'] = combined_df_arrivals['airline'].str.split('\n').str.get(0)
combined_df_departures['airline'] = combined_df_departures['airline'].str.split('\n').str.get(0)
combined_df_arrivals["airline"]


0            Delta Air Lines
1         Southwest Airlines
2          Frontier Airlines
3            Delta Air Lines
4         Chairman Airmotive
                 ...        
398272    Southwest Airlines
398273       Delta Air Lines
398274    Southwest Airlines
398275    Southwest Airlines
398276       Spirit Airlines
Name: airline, Length: 398229, dtype: object

In [42]:
# Encode origin airport, airline One-Hot encoding

# Doing on departures for now

import tensorflow as tf
import numpy as np

# Get the unique origin values from the 'origin' column
origins = combined_df_departures['origin'].unique()

# Create a dictionary to map each origin value to an index
origin_to_index = {origin: index for index, origin in enumerate(origins)}

# Encode the 'origin' column using one-hot encoding
encoded_origins = tf.one_hot(
    tf.constant([origin_to_index[origin] for origin in combined_df_departures['origin']]),
    depth=len(origins)
)

# Convert the encoded origins to a numpy array
encoded_origins = np.array(encoded_origins)

# Get the unique airport code values from the 'airport_code' column
airport_codes = combined_df_departures['airport_code'].unique()

# Create a dictionary to map each airport code value to an index
airport_code_to_index = {airport_code: index for index, airport_code in enumerate(airport_codes)}

# Encode the 'airport_code' column using one-hot encoding
encoded_airport_codes = tf.one_hot(
    tf.constant([airport_code_to_index[airport_code] for airport_code in combined_df_departures['airport_code']]),
    depth=len(airport_codes)
)

# Convert the encoded airport codes to a numpy array
encoded_airport_codes = np.array(encoded_airport_codes)

# Get the unique airline values from the 'airline' column
airlines = combined_df_departures['airline'].unique()

# Create a dictionary to map each airline value to an index
airline_to_index = {airline: index for index, airline in enumerate(airlines)}

# Encode the 'airline' column using one-hot encoding
encoded_airlines = tf.one_hot(
    tf.constant([airline_to_index[airline] for airline in combined_df_departures['airline']]),
    depth=len(airlines)
)

# Convert the encoded airlines to a numpy array
encoded_airlines = np.array(encoded_airlines)

# Print the shape of the encoded arrays
print("Encoded origins shape:", encoded_origins.shape)
print("Encoded airport codes shape:", encoded_airport_codes.shape)
print("Encoded airlines shape:", encoded_airlines.shape)


Encoded origins shape: (316216, 1209)
Encoded airport codes shape: (316216, 28)
Encoded airlines shape: (316216, 407)


## Feature Engineering

Features for number of day ahead delays at Atlanta airport (model 1)
- origin (origin) and destination (airport_code) - invert for departures/arrivals
- airline
- delay percentage 1 day prior
- delay percentage 2 day prior
- delay percentage 3 day prior
- delay percentage 5 day prior
- Federal public holiday
- Season
- Day of the week
- weather (added later)

In [67]:
# Seasonal feature

# Create a new column "season" and initialize with -1
combined_df_arrivals['season'] = -1

# Define the criteria for each season based on month ranges
season_mapping = {
    'Spring': [3, 4, 5],
    'Summer': [6, 7, 8],
    'Autumn': [9, 10, 11],
    'Winter': [12, 1, 2]
}

# Iterate over each row and assign the corresponding season
for index, row in combined_df_arrivals.iterrows():
    month = row['Date'].month
    for season, months in season_mapping.items():
        if month in months:
            # Map the season to numerical values (0, 1, 2, 3)
            combined_df_arrivals.at[index, 'season'] = list(season_mapping.keys()).index(season)
            break

# Print the updated DataFrame
#combined_df_arrivals['season']

In [68]:
# Public holiday feature

# Predefined list of public holidays in the US
public_holidays = ['2023-01-01', '2023-01-16', '2023-02-20', '2023-05-29', '2023-07-04', '2023-09-04', '2023-10-09', '2023-11-10', '2023-11-23', '2023-12-25']

# Convert the "Date" column to datetime type if it's not already
combined_df_arrivals['Date'] = pd.to_datetime(combined_df_arrivals['Date'])

# Create a new column "public_holiday" and initialize with 0
combined_df_arrivals['public_holiday'] = 0

# Set the value to 1 if the date is in the public holidays list
combined_df_arrivals.loc[combined_df_arrivals['Date'].dt.strftime('%Y-%m-%d').isin(public_holidays), 'public_holiday'] = 1

# Print the updated DataFrame
#combined_df_arrivals['public_holiday']

In [66]:
# Delay look back features

# Initialize the delays columns with zeros based on lookback periods
lookback_periods = [1, 2, 3, 5]

# Generate respective columns
for lookback in lookback_periods:
    column_name = f'delays_{lookback}'
    combined_df_arrivals[column_name] = 0

# Get the unique dates in the DataFrame
unique_dates = combined_df_arrivals['Date'].unique()

# Loop through each unique date
for date in unique_dates:
    # Loop through a range of lookback days
    for lookback in [1, 2, 3, 5]:
        # Compute the lookback date
        lookback_date = date - pd.DateOffset(days=lookback)
        
        # Filter the DataFrame for the lookback date
        lookback_data = combined_df_arrivals[combined_df_arrivals['Date'] == lookback_date]
        
        # Compute the delay proportion for the lookback period
        record_count = len(lookback_data)
        delay_count = len(lookback_data[lookback_data['status_encoded'] == 1])
        
        # Compute the delay proportion
        delay_proportion = delay_count / record_count if record_count > 0 else 0
        
        # Update the corresponding rows in the delays columns
        combined_df_arrivals.loc[combined_df_arrivals['Date'] == date, f'delays_{lookback}'] = delay_proportion

# Print the updated DataFrame
#print(combined_df_arrivals["delays_3"])

In [62]:
combined_df_arrivals

,origin,arrive_depart,flight,airline,status,airport_code,Ingestion Date,Airport Name,Date,status_encoded,public_holiday,season,delays_1,delays_2,delays_3,delays_5
0,LAS,12:06 am,DL707,Delta Air Lines,Landed - On-time [+],ATL,2023-05-12 15:50:43.586941,ATL,2023-05-12,0,0,0,0.000000,0.000000,0.000000,0.000000
1,MCO,12:10 am,WN1326,Southwest Airlines,Landed - On-time [+],ATL,2023-05-12 15:50:43.586941,ATL,2023-05-12,0,0,0,0.000000,0.000000,0.000000,0.000000
2,MIA,12:10 am,F92337,Frontier Airlines,Landed - Delayed [+],ATL,2023-05-12 15:50:43.586941,ATL,2023-05-12,1,0,0,0.000000,0.000000,0.000000,0.000000
3,JFK,12:13 am,DL1248\nAZ3401\nKL5031\nUX3424\nVS4817\nWS7756,Delta Air Lines,Landed - On-time [+],ATL,2023-05-12 15:50:43.586941,ATL,2023-05-12,0,0,0,0.000000,0.000000,0.000000,0.000000
4,DAL,12:27 am,CHR10,Chairman Airmotive,Landed - On-time [+],ATL,2023-05-12 15:50:43.586941,ATL,2023-05-12,0,0,0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398272,BWI,11:30 pm,WN1895,Southwest Airlines,Landed - Delayed [+],TPA,2023-06-25 03:52:10.522540,TPA,2023-06-25,1,0,1,0.372365,0.354923,0.313736,0.257334
398273,MSP,11:37 pm,DL1624\nKL5294\nVS3418\nWS7331,Delta Air Lines,Landed - On-time [+],TPA,2023-06-25 03:52:10.522540,TPA,2023-06-25,0,0,1,0.372365,0.354923,0.313736,0.257334
398274,DAL,11:40 pm,WN204,Southwest Airlines,Landed - On-time [+],TPA,2023-06-25 03:52:10.522540,TPA,2023-06-25,0,0,1,0.372365,0.354923,0.313736,0.257334
398275,MKE,11:45 pm,WN1052,Southwest Airlines,Landed - On-time [+],TPA,2023-06-25 03:52:10.522540,TPA,2023-06-25,0,0,1,0.372365,0.354923,0.313736,0.257334


## Model 1 - day ahead delays at Atlanta airport

## EDA
- Delays per day time series
- Rank airlines by proportion of delays
- Rank airports by proportion of delays


In [36]:
import matplotlib.pyplot as plt
import pandas as pd


# Group the data by month and calculate the sum of delays for arrivals and departures
arrivals_delays_per_month = combined_df_arrivals.groupby(pd.Grouper(key='Date', freq='M'))['status'].sum()
departures_delays_per_month = combined_df_departures.groupby(pd.Grouper(key='Date', freq='M'))['status'].sum()

# Create a line plot for arrivals delays
plt.figure(figsize=(12, 6))
plt.plot(arrivals_delays_per_month.index, arrivals_delays_per_month, label='Arrivals', color='blue')
plt.xlabel('Month')
plt.ylabel('Delays')
plt.title('Arrivals Delays per Month')
plt.legend()
plt.show()

# Create a line plot for departures delays
plt.figure(figsize=(12, 6))
plt.plot(departures_delays_per_month.index, departures_delays_per_month, label='Departures', color='green')
plt.xlabel('Month')
plt.ylabel('Delays')
plt.title('Departures Delays per Month')
plt.legend()
plt.show()

## Data encoding

### ML

Aim of each model
- predict day ahead delays for all airports as aggregate
- predict day ahead delays for each airport
- predict day ahead delays for each airline

In [ ]:
import tensorflow as tf
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('flight_data.csv')

# Preprocess the data
# Assuming the DataFrame has a 'timestamp' column and a 'status' column
timestamps = pd.to_datetime(df['timestamp'])
statuses = df['status'].map({'on time': 0, 'delayed': 1})

# Convert the data to TensorFlow tensors
timestamps_tensor = tf.constant(timestamps.values, dtype=tf.float32)
statuses_tensor = tf.constant(statuses.values, dtype=tf.float32)

# Define the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(timestamps_tensor, statuses_tensor, epochs=10, batch_size=32)

# Make predictions
predictions = model.predict(timestamps_tensor)

# Perform further analysis or evaluation as needed
